# Tests MSSQL Python

Frist create a .env file:

```env
MSSQL_HOST="..."
MSSQL_PORT=1433
MSSQL_DATABASE="..."
MSSQL_SCHEMA="dbo"
MSSQL_USERNAME="..."
MSSQL_PASSWORD="..."
```

In [1]:
%reload_ext dotenv
%dotenv

import os

config = {
    "host": os.getenv("MSSQL_HOST"),
    "port": int(os.getenv("MSSQL_PORT") or 0),
    "database": os.getenv("MSSQL_DATABASE"),
    "schema": os.getenv("MSSQL_SCHEMA") or "dbo",
    "username": os.getenv("MSSQL_USERNAME"),
    "password": os.getenv("MSSQL_PASSWORD"),
}

## PyODBC

**Microsoft ODBC Driver for SQL Server is required before to install pyodbc**.

Please follow the instructions to install the driver for your OS. See the [docs](https://learn.microsoft.com/en-us/sql/connect/odbc/microsoft-odbc-driver-for-sql-server) for details.

To install run this command:
```bash
poetry add pyodbc
```

In [ ]:
import pyodbc

sql_server_drivers = list(filter(lambda x: "SQL Server" in x, pyodbc.drivers()))
conn_str = (
    f"DRIVER={{{sql_server_drivers[0]}}};"
    f'SERVER=tcp:{config["host"]};PORT={config["port"]};'
    f'DATABASE={config["database"]};'
    f'UID={config["username"]};'
    f'PWD={config["password"]}'
)
display(conn_str)

with pyodbc.connect(conn_str) as conn:
    with conn.cursor() as cursor:
        rows = cursor.execute(
            f"""
            SELECT
                *
            FROM
                INFORMATION_SCHEMA.TABLES
            WHERE
                TABLE_CATALOG='{config['database']}'
                AND TABLE_SCHEMA='dbo'
                AND TABLE_TYPE='BASE TABLE';
            """
        )
        display(rows.fetchall())

## PyTDS

To install run this command:

```bash
poetry add python-tds
```

If you want to use TLS you should also install pyOpenSSL package:

```bash
poetry add pyOpenSSL
```

For a better performance install bitarray package too:

```bash
poetry add bitarray
```

To use Kerberos on non-Windows platforms (experimental) install kerberos package:

```bash
poetry add kerberos
```

## MiniAB

**miniab** requires duckdb and pyarrow and they should be installed before to use it.

To install duckdb and pyarrow run this command:

```bash
poetry add duckdb pyarrow
```

In [10]:
import duckdb
import miniab as mab

display(mab.get_available_connectors())

result = mab.get_source("source-mssql", config=config, streams="*").read()

tteam_tbl = result.to_arrow("TTeam")
duckdb.sql("SELECT * FROM tteam_tbl;").show()

['source-snowflake', 'source-mssql']

Sync Progress: MsSqlSource -> DuckdbCache
Started reading from source at 13:01:56
 * Finished reading from source at 13:01:56.
Sync completed at 13:01:56. Total time elapsed: 0:00:00.049717
┌────────┬────────────────┬────────────┬───────────┐
│ TeamID │    TeamName    │ JobTitleID │ VersionID │
│ int64  │    varchar     │   int64    │   int64   │
├────────┼────────────────┼────────────┼───────────┤
│      2 │ EMERALD        │          5 │         1 │
│      3 │ RUBY-PROVINCES │          5 │         1 │
│      4 │ SAPHIRE        │          5 │         1 │
│      5 │ TOPAZ          │          5 │         1 │
│      6 │ JADE           │          5 │         1 │
│      7 │ JASPER         │          5 │         1 │
│      8 │ RUBY           │          5 │         1 │
│     10 │ PKAS           │          8 │         1 │
│     22 │ KAS            │          4 │         1 │
│     24 │ HKAS           │          7 │         1 │
│      · │  ·             │          · │         · │
│      · │  ·  

## PyAirByte

PyAirByte requires docker and it should be installed before to use it.

To install PyAirByte run this command:

```bash
poetry add airbyte
```

In [ ]:
import airbyte as ab
import duckdb

display(ab.get_available_connectors())

result = mab.get_source("source-mssql", config=config, streams="*").read()

tteam_tbl = result.to_arrow("TTeam")
duckdb.sql("SELECT * FROM tteam_tbl LIMIT 10;").show()